In [1]:
import numpy as np
import pandas as pd
import os
from collections import Counter

dpath = "/home/nikita/PycharmProjects/taxi/processed_data/TwoDstat.csv"
twoD = pd.read_csv(dpath, index_col="Zone")
cellSum = twoD.sum(axis=1)
cellSum.iloc[[1225 + i for i in range(10)]]

Zone
1226      3991
1227      2662
1228    120597
1229    236281
1230    312521
1231    476106
1232    489489
1233    552503
1234    394985
1235    265118
dtype: int64

Кол-во нулевых

In [2]:
zero_zones = Counter(cellSum)[0]
print(zero_zones)

1283


Построим карту Нью-Йорка

In [3]:
import folium
folium_map = folium.Map(location=[40.738, -73.98],
                        zoom_start=10,
                        tiles="CartoDB dark_matter")


Подготовим json для визуализации

In [4]:
import json

res_json_dict = {
    "type": "FeatureCollection",
    "features": []
}

dpath = "/home/nikita/PycharmProjects/taxi/raw_data/regions.csv"
f = open(dpath)
l = f.readline()
features = []

for l in f:
    row = l.split(";")
    f = {
        "type": "Feature",
        "id": "test",
        "properties": {
            "name": "test"
        },
        "geometry": {
            "type": "Polygon",
            "coordinates": []
        }
    }
    f["id"] = int(row[0])
    for i in range(len(row)):
        row[i] = float(row[i])
    coordinates = []
    coordinates.append([row[1], row[3]])
    coordinates.append([row[1], row[4]])
    coordinates.append([row[2], row[4]])
    coordinates.append([row[2], row[3]])
    f["geometry"]["coordinates"] = [coordinates]
    features.append(f)

res_json_dict["features"] = features
res_json = json.dumps(res_json_dict)
resfile = "NYheo.json"
f = open(resfile, 'w')
f.write(res_json)

556636

Построим карту с ячейками цветом, отображающим максимальное кол-во поездок

In [5]:
dpath = "/home/nikita/PycharmProjects/taxi/processed_data/TwoDstat.csv"
twoD = pd.read_csv(dpath, index_col="Zone")
d = twoD.sum(axis=1).values.reshape((twoD.shape[0], 1))
z = np.array(range(1, len(d) + 1)).reshape((len(d), 1))
dz = np.concatenate((z, d), axis=1)

data = pd.DataFrame(dz, columns=['Zone', 'Rides'])
#
geo_data = "/home/nikita/PycharmProjects/taxi/week2/NYheo.json"
# Initialize the map:
m = folium.Map(location=[40.738, -73.98],
                        zoom_start=10,)

# Add the color for the chloropleth:
m.choropleth(
    geo_data=geo_data,
    name='choropleth',
    data=data,
    columns=['Zone', 'Rides'],
    key_on='feature.id',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Rides number'
)
folium.LayerControl().add_to(m)
est = (40.7484, -73.9857)
folium.Marker(location=est, popup="Empire State Building").add_to(m)
# Save to html
m.save('AllRides.html')

### Статическую картинку я не смог сделать, обе html-ки в архиве

Отфильтруем поездки с менне чем 5 поездками

In [6]:
more_than5zones = []
dpath = "/home/nikita/PycharmProjects/taxi/processed_data/TwoDstat.csv"
twoD = pd.read_csv(dpath, index_col="Zone")
TwoDMean = twoD.mean(axis=1).values.reshape((twoD.shape[0], 1))
more_than5ids = []
more_than5values = []
for i in range(TwoDMean.shape[0]):
    rides_per_hour = TwoDMean[i][0]
    if rides_per_hour >= 5:
        more_than5zones.append(features[i])
        more_than5ids.append(features[i]["id"])
        more_than5values.append(rides_per_hour)
        
        
res_json_dict["features"] = more_than5zones
res_json = json.dumps(res_json_dict)
resfile = "RidesPerHour.json"
f = open(resfile, 'w')
f.write(res_json)

22735

In [7]:
data = pd.DataFrame({'Zone': more_than5ids,'Rides' : more_than5values })
geo_data = "/home/nikita/PycharmProjects/taxi/RidesPerHour.json"
# Initialize the map: 40.6892° N, 74.0445
meanMap = folium.Map(location=[40.6892, -74.0445],
                        zoom_start=10,)

# Add the color for the chloropleth:
meanMap.choropleth(
    geo_data=geo_data,
    name='choropleth',
    data=data,
    columns=['Zone', 'Rides'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Rides number'
)
folium.LayerControl().add_to(meanMap)
# 40.6892° N, 74.0445
marker = folium.Marker(location=(40.6892, -74.0445), popup="Statue Of Liberty")
marker.add_to(meanMap)
# Save to html
meanMap.save('RidesPerHour.html')

Одна из ячеек полностью находится в воде.